In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/arcane/')
import pandas as pd

os.environ['PATH'] = '/Users/ravi.tej/anaconda3/envs/bertopicenv/bin:/Users/ravi.tej/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin'

In [2]:
from sql.articles.MongoDBArticle import MongoDBArticle

In [3]:
from datetime import datetime

In [22]:
def fetch_all_articles(limit=None):
    query = {
    # "is_premium_article": False,
    "content_type": 'ARTICLE',
    # "cleaned_text": {"$exists": True, "$ne": None},
    "published_time": {'$gt': "2024-02-28"}
    }
    collection = MongoDBArticle.get_collection()
    if isinstance(limit, int):
        documents = list(collection.find(query).limit(limit))
    else:
        documents = list(collection.find(query))
    for doc in documents:
        doc['article_id'] = str(doc['_id'])
        doc['source_id'] = str(doc['source_id'])
    return documents

In [23]:
docs1 = fetch_all_articles()

In [24]:
len(docs1)

37086

In [25]:
df2 = pd.DataFrame(docs1)

In [7]:
df = pd.DataFrame(docs)

In [8]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [9]:
with PostgresDatabaseOperation() as cursor:
    sql = "SELECT DISTINCT article_id, created_at FROM embeddings WHERE created_at > '2024-02-28'"
    cursor.execute(sql)
    results = cursor.fetchall()

In [10]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT DISTINCT source_id, source_name FROM source_id_characteristics'
    cursor.execute(sql)
    source_results = cursor.fetchall()

In [11]:
emb_df = pd.DataFrame(results, columns = ['article_id', 'created_at'])

In [12]:
source_df = pd.DataFrame(source_results, columns = ['source_id', 'source_name'])

In [13]:
source_df

,source_id,source_name
0,6560b6109952a16bbc397227,economist
1,650046bd005149c49201269f,live-mint
2,655598ab96fb32eef21a562c,business-insider
3,6512cdcad01a9c8e86263e05,economic-times
4,6560a5bddf69601d3e2ce0c1,bbc
5,65768d2e7c73e77fdb13a9b3,inc42
6,652d53256a2736f06f46cfcf,wint-wealth
7,65559d0c96fb32eef21a562f,business-world
8,65291b1e9a2fbc229e5f29c9,zerodha-varsity
9,65558d1f672118037c541088,navi


In [14]:
len(emb_df)

38175

In [15]:
df1 = pd.merge(df, source_df, how = 'left', on = 'source_id')

In [26]:
df3 = pd.merge(df2, source_df, how = 'left', on = 'source_id')

In [16]:
df1

,_id,url,published_time,last_updated_time,source_id,image_url,authors,reactions,content_type,_class,cleaned_text,is_premium_article,short_description,tags,title,ai_generated_info,article_id,source_name
0,65dea1a8b25990560f7df915,https://www.livemint.com/market/live-blog/delt...,2024-02-28T08:10:41+05:30,2024-02-28T08:10:41+05:30,650046bd005149c49201269f,https://www.livemint.com/lm-img/img/2023/05/04...,[Livemint],{},ARTICLE,com.wintWealth.insight.common.models.entities....,Delta Corp Share Price Today : Delta Corp's st...,False,"Delta Corp stock price went down today, 28 Feb...","[Delta Corp Share Price Live, Delta Corp Stock...",Delta Corp Share Price Live blog for 28 Feb 20...,{'summary': {'OpenHermes_WatermelonUvulaMarigo...,65dea1a8b25990560f7df915,live-mint
1,65dea1aab25990560f7df917,https://www.livemint.com/market/live-blog/reli...,2024-02-28T08:09:19+05:30,2024-02-28T08:09:19+05:30,650046bd005149c49201269f,https://www.livemint.com/lm-img/img/2023/05/04...,[Livemint],{},ARTICLE,com.wintWealth.insight.common.models.entities....,Reliance Power Share Price Today : On the last...,False,"Reliance Power stock price went down today, 28...","[Reliance Power Share Price Live, Reliance Pow...",Reliance Power Share Price Live blog for 28 Fe...,{'summary': {'OpenHermes_WatermelonUvulaMarigo...,65dea1aab25990560f7df917,live-mint
2,65dea1aab25990560f7df918,https://www.moneycontrol.com/news/business/mar...,2024-02-28T01:52:37+0000,2024-02-28T02:43:40+0000,6560af039952a16bbc397224,https://images.moneycontrol.com/static-mcnews/...,[Rakesh Patil],{},ARTICLE,com.wintWealth.insight.common.models.entities....,"Juniper Hotels likely to see muted debut, grey...",False,Sensex Today | Stock Market LIVE Updates: US s...,"[sensex, sensex today, bse sensex live, stock ...",Stock Market LIVE Updates: GIFT Nifty indicate...,{'summary': {'OpenHermes_WatermelonUvulaMarigo...,65dea1aab25990560f7df918,money-control
3,65dea1a8b25990560f7df914,https://www.livemint.com/market/live-blog/asia...,2024-02-28T08:11:35+05:30,2024-02-28T08:11:35+05:30,650046bd005149c49201269f,https://www.livemint.com/lm-img/img/2023/05/04...,[Livemint],{},ARTICLE,com.wintWealth.insight.common.models.entities....,Asian Paints Share Price Today : Asian Paints ...,False,"Asian Paints stock price went down today, 28 F...","[Asian Paints Share Price Live, Asian Paints S...",Asian Paints Share Price Live blog for 28 Feb ...,{'summary': {'OpenHermes_WatermelonUvulaMarigo...,65dea1a8b25990560f7df914,live-mint
4,65dea1acb25990560f7df924,https://www.livemint.com/market/live-blog/ange...,2024-02-28T08:10:29+05:30,2024-02-28T08:10:29+05:30,650046bd005149c49201269f,https://www.livemint.com/lm-img/img/2023/05/04...,[Livemint],{},ARTICLE,com.wintWealth.insight.common.models.entities....,Angel One Share Price Today : Angel One's stoc...,False,"Angel One stock price went down today, 28 Feb ...","[Angel One Share Price Live, Angel One Stock P...",Angel One Share Price Live blog for 28 Feb 202...,{'summary': {'OpenHermes_WatermelonUvulaMarigo...,65dea1acb25990560f7df924,live-mint
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36406,65e7336941eea86ba330c04e,https://www.livemint.com/market/stock-market-n...,2024-03-05T18:46:59+05:30,2024-03-05T18:46:59+05:30,650046bd005149c49201269f,https://www.livemint.com/lm-img/img/2024/03/05...,[Vaamanaa Sethi],{},ARTICLE,com.wintWealth.insight.common.models.entities....,FMCG company Gopal Snacks is all set to launch...,False,The shares of Gopal Snacks IPO are available a...,"[Gopal Snacks IPO, Gopal Snacks IPO details, G...",Gopal Snacks IPO to open tomorrow; Here are 10...,NaN,65e7336941eea86ba330c04e,live-mint
36407,65e7336941eea86ba330c051,https://www.livemint.com/market/stock-market-n...,2024-03-05T20:17:52+05:30,2024-03-05T20:17:52+05:30,650046bd005149c49201269f,https://www.livemint.com/lm-img/img/2024/03/05...,[Rajendra Saxena],{},ARTICLE,com.wintWealth.insight.common.models.entities....,US stocks opened lower on Tuesday ahead o

In [17]:
datetime.strptime('2024-02-28T15:05:00+0000','%Y-%m-%dT%H:%M:%S%z').date()

datetime.date(2024, 2, 28)

In [33]:
100 * df3[df3.source_name != 'yahoo-finance'].groupby('source_name')['source_name'].count()/len(df3[df3.source_name != 'yahoo-finance'])

source_name
bbc                   1.045236
business-standard    21.350240
economic-times        7.008225
finshots              0.188485
groww                 0.017135
inc42                 3.735435
investopedia          5.089102
live-mint            27.159013
money-control        15.438657
reuters              18.968472
Name: source_name, dtype: float64

In [37]:
df3[df3.source_name != 'yahoo-finance'].groupby('source_name')['source_name'].count() - df1[df1.source_name != 'yahoo-finance'].groupby('source_name')['source_name'].count()

source_name
bbc                    0
business-standard    103
economic-times       301
finshots               0
groww                  0
inc42                  0
investopedia           0
live-mint            193
money-control         78
reuters                0
Name: source_name, dtype: int64

In [36]:
df1[df1.source_name != 'yahoo-finance'].groupby('source_name')['source_name'].count()

source_name
bbc                    61
business-standard    1143
economic-times        108
finshots               11
groww                   1
inc42                 218
investopedia          297
live-mint            1392
money-control         823
reuters              1107
Name: source_name, dtype: int64

In [34]:
100 * df1[df1.source_name != 'yahoo-finance'].groupby('source_name')['source_name'].count()/len(df1[df1.source_name != 'yahoo-finance'])

source_name
bbc                   1.181941
business-standard    22.146871
economic-times        2.092618
finshots              0.213137
groww                 0.019376
inc42                 4.223988
investopedia          5.754699
live-mint            26.971517
money-control        15.946522
reuters              21.449332
Name: source_name, dtype: float64

In [27]:
df3['dt'] = df3['published_time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%dT%H:%M:%S%z').date())

ValueError: time data '2024-03-06' does not match format '%Y-%m-%dT%H:%M:%S%z'

In [19]:
pd.options.display.max_rows = 100

In [20]:
df1.groupby(['source_name'])['article_id'].count()

source_name
bbc                     61
business-standard     1143
economic-times         108
finshots                11
groww                    1
inc42                  218
investopedia           297
live-mint             1392
money-control          823
reuters               1107
yahoo-finance        31250
Name: article_id, dtype: int64

In [21]:
df1.groupby(['dt','source_name'])['article_id'].count()

dt          source_name      
2024-02-28  bbc                     9
            business-standard     186
            economic-times         10
            finshots                1
            inc42                  17
            investopedia           40
            live-mint             192
            money-control         163
            reuters               146
            yahoo-finance        4201
2024-02-29  bbc                    10
            business-standard     195
            economic-times         13
            finshots                1
            groww                   1
            inc42                  83
            investopedia           34
            live-mint             197
            money-control         109
            reuters               190
            yahoo-finance        5263
2024-03-01  bbc                    12
            business-standard     194
            economic-times         12
            finshots                2
            inc42   

In [24]:
df

,_id,url,source_id,authors,reactions,content_type,_class,cleaned_text,image_url,is_premium_article,last_updated_time,published_time,short_description,tags,title,ai_generated_info,article_id
0,65df9f2db25990560f7e0773,https://finance.yahoo.com/news/bill-gates-rece...,6560ac8b9952a16bbc397223,[CNW Group],{},ARTICLE,com.wintWealth.insight.common.models.entities....,"BHUBANESWAR, India , Feb. 28, 2024 /CNW/ -- Le...",https://media.zenfs.com/en/cnwgroup.com/b93b46...,False,2024-02-28T15:05:00+0000,2024-02-28T15:05:00+0000,"Leading global philanthropist, Mr. Bill Gates,...","[Bill Gates, Bill & Melinda Gates Foundation, ...",Bill Gates Receives the Prestigious KISS Human...,{'summary': {'OpenHermes_WatermelonUvulaMarigo...,65df9f2db25990560f7e0773
1,65df9f1fb25990560f7e06ae,https://finance.yahoo.com/news/1-european-parl...,6560ac8b9952a16bbc397223,[Reuters],{},ARTICLE,com.wintWealth.insight.common.models.entities....,(Adds extra comment from EU in paragraph 2) BR...,https://s.yimg.com/cv/apiv2/social/images/yaho...,False,2024-02-28T15:29:24+0000,2024-02-28T15:29:24+0000,Amazon.com lobbyists have been banned from the...,"[European Parliament, employment committee, Am...",UPDATE 1-European Parliament bans Amazon lobby...,{'summary': {'OpenHermes_WatermelonUvulaMarigo...,65df9f1fb25990560f7e06ae
2,65df9f2db25990560f7e0776,https://finance.yahoo.com/news/argentina-18-bi...,6560ac8b9952a16bbc397223,[Jonathan Gilbert and Bob Van Voris],{},ARTICLE,com.wintWealth.insight.common.models.entities....,"(Bloomberg) -- Burford Capital, which is tryin...",https://s.yimg.com/ny/api/res/1.2/hZpQLuWOOWXV...,False,2024-02-28T18:53:40+0000,2024-02-28T18:53:40+0000,"(Bloomberg) -- Burford Capital, which is tryin...","[Bloomberg, Argentina, Burford Capital, curren...",Argentina’s $18 Billion Chinese FX Line Eyed f...,{'summary': {'OpenHermes_WatermelonUvulaMarigo...,65df9f2db25990560f7e0776
3,65df9f30b25990560f7e07a2,https://finance.yahoo.com/news/safir-shamsi-ro...,6560ac8b9952a16bbc397223,[PR Newswire],{},ARTICLE,com.wintWealth.insight.common.models.entities....,"Safir Shamsi , renowned real estate agent at R...",https://media.zenfs.com/en/prnewswire.com/b458...,False,2024-02-28T18:00:00+0000,2024-02-28T18:00:00+0000,"Safir Shamsi, a distinguished real estate agen...","[Safir Shamsi, real estate agent, Safir, resid...",Safir Shamsi of Rodeo Realty Inc. Achieves Rem...,{'summary': {'OpenHermes_WatermelonUvulaMarigo...,65df9f30b25990560f7e07a2
4,65e063e37825a96cf7dc27ff,https://finance.yahoo.com/news/private-equity-...,6560ac8b9952a16bbc397223,[editorial-team@simplywallst.com (Simply Wall ...,{},ARTICLE,com.wintWealth.insight.common.models.entities....,Key Insights Trevi Therapeutics' significant p...,https://media.zenfs.com/en/simply_wall_st__316...,False,2024-02-29T10:36:55+0000,2024-02-29T10:36:55+0000,Key Insights Trevi Therapeutics' significant p...,"[private equity firms, shareholders]",private equity firms who own 35% along with in...,{'summary': {'OpenHermes_WatermelonUvulaMarigo...,65e063e37825a96cf7dc27ff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22124,65df9f1eb25990560f7e06a8,https://finance.yahoo.com/news/koning-health-a...,6560ac8b9952a16bbc397223,[PR Newswire],{},ARTICLE,com.wintWealth.insight.common.models.entities....,"DUBAI , UAE , Feb. 28, 2024 /PRNewswire/ -- Ko...",https://media.zenfs.com/en/prnewswire.com/6ba9...,False,2024-02-28T17:12:00+0000,2024-02-28T17:12:00+0000,"Koning Health, a pioneering medical imaging co...","[Ministry of Health, breast imaging, Koning He...",Koning Health Achieves UAE Regulatory Clearanc...,{'summary': {'OpenHermes_WatermelonUvulaMarigo...,65df9f1eb25990560f7e06a8
22125,65df9f2db25990560f7e077a,https://finance.yahoo.com/news/mortgage-rates-...,6560ac8b9952a16bbc397223,[Megan Henney],{},ARTICLE,com.wintWealth.insight.common.models.entities....,A key measure of home-purchase applications fe...,https://s.yimg.com/ny/api/res/1.2/KxfY_dLWeLP9...,False,2024-02-28T15:41:04+0000,2024-